In [28]:
# Google Drive 연동

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
!pip install konlpy

In [30]:
import os
import re
import sys
import csv
import random
import numpy as np
import pandas as pd
from IPython.display import clear_output
from konlpy.tag import Okt
from keras.layers import Dense, LSTM, Input, Embedding, Dropout, Bidirectional, TimeDistributed
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.models import Model, load_model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

root_dir = '/content/gdrive/My Drive/ITWILL/Project/'
csv_list = os.listdir(root_dir + 'data/')
print(csv_list)

['data.csv', 'data_underground500.csv', 'tokens_underground500.csv', 'tokens_underground100.csv', 'data_underground_100.csv', 'data_underground500.gsheet']


In [31]:
df = pd.read_csv(root_dir + 'data/' + 'data_underground500.csv')

df.head()

,ranking,title,artist,album,likes,date,lyrics
0,1,19금\n변태 (Prod. PAAD),PAAD,변태,3,2020.02.25,가사가 없습니다
1,2,GPS,Yonge Jaundice (영 잔디스),LATE NITE AT 031,4,2020.02.25,Where’s myself<br/>몰라<br/>지구 반대쯤<br/>Somebody ...
2,3,WASHING FUNCTION (Feat. Xyro),OB DIME,WASHING FUNCTION,4,2020.02.25,꿈 꾸네 딴 나라의 밤<br/>꿈 꿨지 그 도시의 낮<br/>서울과는 다를 것 같아...
3,4,Juice (Feat. Band Westwood),Life20house (라이프투애니하우스),Juice,10,2020.02.25,쏟아지는 Juice 흘러넘쳐버렸어 <br/>널 보는 내 마음 한계를 넘었어 <br/...
4,5,난 너의 하늘이 되기로 했다,하밍구,난 너의 하늘이 되기로 했다,6,2020.02.25,ㅎㅁ9<br/>헝클어진 머리<br/>어제 먹다 남은 피자<br/>더부룩하지만<br/...


In [32]:
df_ = df['lyrics']

len(df_)

500

In [33]:
index = []
for i in range(len(df_)) :
    if df_.iloc[i] == '가사가 없습니다' : index.append(i)

print(index)

[0, 59, 83, 90, 100, 110, 153, 171, 172, 183, 189, 197, 220, 294, 376, 385, 406, 417, 429]


In [34]:
df_.drop(index, inplace = True)

len(df_)

481

In [0]:
start_story = '| ' * 20
text = ''

for i in range(int(len(df_)/10)) :
    text += start_story 
    text += df_.iloc[i]

In [36]:
text_length = len(text)
text_length

70468

In [37]:
text

"| | | | | | | | | | | | | | | | | | | | Where’s myself<br/>몰라<br/>지구 반대쯤<br/>Somebody use ur gps on me<br/>Somebody use ur gps on me<br/>Where’s ur self<br/>July<br/>Summer memories<br/>I gotta use my gps on u<br/>I gotta use my gps on u<br/>그때의 나로 대려다줄 독백<br/>Pray to my god but I aint obey<br/>십자가 앞에 내 모습은 초췌<br/>죄를 씻어도 내 온몸에는 satan<br/>늪에 빠진듯<br/>im drowning im drowning<br/>My darling<br/>U doing ok<br/>혼자가 싫어<br/>I try to fall in I fall in<br/>Love but u already know I cant<br/>익숙해 being loner<br/>Lighting over lunar<br/>밝아보여 brighter than me<br/>I want no one on me<br/>하늘이 밝아질때<br/>밝았던 기억을 떠올릴께<br/>I remind u when I feel pain<br/>I can’t find u I lost in the hell<br/>Where’s myself<br/>몰라<br/>지구 반대쯤<br/>Somebody use ur gps on me<br/>Somebody use ur gps on me<br/>Where’s ur self<br/>July<br/>Summer memories<br/>I gotta use my gps on u<br/>I gotta use my gps on u<br/>떠나줘<br/>내 기억속에 남은 사진들만<br/>선명하게 남을때<br/>태워도<br/>타지 않는 잠수성의 표면처럼<br/>그때로난 잠기네<br/>Where r u where u<br/>Where the hell

In [38]:
text = re.sub('<br/>', '##', text)
text = re.sub('[A-Za-z]', '', text)
text = re.sub('\n', '', text)
text = re.sub('#+', '##', text)
text = re.sub('’', '', text)
text = re.sub('##', ' br ', text)
text = re.sub(' +', ' ', text)
text = re.sub('[br ]{2,}', ' br ', text)

text

"| | | | | | | | | | | | | | | | | | | | br 몰라 br 지구 반대쯤 br 그때의 나로 대려다줄 독백 br 십자가 앞에 내 모습은 초췌 br 죄를 씻어도 내 온몸에는 br 늪에 빠진듯 br 혼자가 싫어 br 익숙해 br 밝아보여 br 하늘이 밝아질때 br 밝았던 기억을 떠올릴께 br 몰라 br 지구 반대쯤 br 떠나줘 br 내 기억속에 남은 사진들만 br 선명하게 남을때 br 태워도 br 타지 않는 잠수성의 표면처럼 br 그때로난 잠기네 br | | | | | | | | | | | | | | | | | | | | 꿈 꾸네 딴 나라의 밤 br 꿈 꿨지 그 도시의 낮 br 서울과는 다를 것 같아서 br 머리 위 비행기 꼬리만 br 따라가네 내 눈동자는 br 언젠가는 타보겠지만 br 티켓 값을 벌은 뒤에 br 첨 찍어 볼 여권 사진엔 br 상승과 추락이 가늠 안될 br 표정 한번을 남긴 뒤 br 느낄 쓴 맛 있는거 br 싹 털어 가거든 아무리 br 미쳤다 해도 뒷날은 br 내 마음의 여유를 좀먹지 br 아무래도 맘 편한 나라는 없나 보네 br 나 같은 놈에게는 br 그래 종일 음악 얘기 br 폐는 한계를 맞지 br 붕 뜬 시간을 잡기엔 br 거드름 피운 날 많으니 br 급해졌어 변명 안 해 우스워져 br 느낀 것 아직 없는데 추운 겨울 br 내려놔 반쯤 떠도 나쁘지 않네 br 서울은 꽤 멋져 단지 우리는 아직 br 비우지 못한 게 많으니 br 새로 채워 덜어내 br 됐어 다 비운 다음엔 br 굳이 떠날 필요가 없어 br 굳이 떠날 필요가 없어 br 나 역시 멀리 도망 가자고 br 창문을 열면 푸른 바다고 br 잘 웃고 매너 좋은 사람들 br 원한건 그게 전부 다 라고 br 난 떠나야 해 자유 찾아서 br 계속해 되뇌였던 착각들 br 이곳엔 자유로운 나란 없고 br 자유로운 마음만 존재한단걸 br 늦게 나마 알았어 br 나와 똑같은 검은 초상 br 어쩜 늦지 않았어 br 널 벗어나 멀리 도망 br 바꿔 내 시야란거 br 자유로움과 같이

In [0]:
f = open(root_dir + "lyrics.txt", "w")
f.write(text)
f.close()

In [0]:
okt = Okt()
def tokenize(text) :
    tokens = okt.morphs(text, stem=True)
    return tokens

In [41]:
tokened_text = tokenize(text)

print(tokened_text)

['|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', 'br', '모르다', 'br', '지구', '반대', '쯤', 'br', '그때', '의', '나로', '대다', '독백', 'br', '십자가', '앞', '에', '내', '모습', '은', '초췌', 'br', '죄', '를', '씻다', '내', '온몸', '에는', 'br', '늪', '에', '빠지다', 'br', '혼자', '가', '싫다', 'br', '익숙하다', 'br', '밝다', '보이다', 'br', '하늘', '이', '밝아지다', '때', 'br', '밝다', '기억', '을', '떠올리다', 'br', '모르다', 'br', '지구', '반대', '쯤', 'br', '떠나다', 'br', '내', '기', '억', '속', '에', '남다', '사진', '들', '만', 'br', '선명하다', '남', '을', '때', 'br', '태우다', 'br', '타지', '않다', '잠수', '성의', '표면', '처럼', 'br', '그때', '로', '난', '잠기', '네', 'br', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '꿈', '꾸다', '따다', '나라', '의', '밤', 'br', '꿈', '끄다', '그', '도시', '의', '낮', 'br', '서울', '과는', '다르다', '것', '같다', 'br', '머리', '위', '비행기', '꼬리', '만', 'br', '따라가다', '내', '눈동자', '는', 'br', '언젠가', '늘다', '타', '보다', 'br', '티켓', '값', '을', '벌다', '뒤', '에', 'br', '첨', '찍다', '볼', '여권', '사진', '엔',

In [42]:
total_words = sorted(list(set(text)))
len(total_words)

890

In [43]:
tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

print(token_list)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 67, 1, 724, 725, 1, 1220, 1221, 1222, 1223, 1, 726, 475, 5, 1224, 1225, 1, 1226, 1227, 5, 1228, 1, 1229, 1230, 1, 330, 51, 1, 1231, 1, 1232, 1, 266, 1233, 1, 1234, 727, 1235, 1, 67, 1, 724, 725, 1, 1236, 1, 5, 1237, 476, 1238, 1, 1239, 1240, 1, 1241, 1, 1242, 161, 1243, 1244, 1, 1245, 1246, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 331, 1247, 728, 1248, 332, 1, 331, 1249, 24, 1250, 1251, 1, 1252, 477, 69, 729, 1, 333, 193, 1253, 1254, 1, 1255, 5, 1256, 1, 1257, 1258, 1, 730, 1259, 1260, 97, 1, 1261, 1262, 731, 1263, 1264, 1, 1265, 1266, 732, 1267, 1, 1268, 1269, 1270, 267, 1, 1271, 733, 734, 1272, 1, 735, 736, 1273, 194, 1, 1274, 125, 1275, 1, 5, 737, 1276, 1277, 1, 738, 1278, 1279, 1280, 1281, 1282, 1, 26, 126, 1283, 1, 46, 1284, 334, 478, 1, 1285, 739, 740, 1, 1286, 479, 98, 1287, 1, 1288, 1289, 9, 480, 1, 1290, 1291, 31, 10, 1292, 1, 1293, 69, 38, 741, 742, 743, 1, 744, 745, 746, 747, 748, 1, 749, 7

In [44]:
def generate_sequences(token_list, step):
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i : i+seq_length])
        y.append(token_list[i+seq_length])
        
    y = np_utils.to_categorical(y, num_classes = total_words) 

    num_seq = len(X)        
    print('시퀀스 개수:', num_seq, '\n')
    
    return X, y, num_seq


step = 1
seq_length = 20
X, y, num_seq = generate_sequences(token_list, step)

시퀀스 개수: 13185 



In [0]:
X = np.array(X)
y = np.array(y)

In [47]:
print('Running model...')

n_units = 256
embedding_size = 100

text_in = Input(shape = (None,))
x = Embedding(total_words, embedding_size)(text_in)
x = LSTM(units = 3, return_sequences = True)(x)
x = Bidirectional(LSTM(3))(x)
x = Dropout(rate = 0.2)(x)
text_out = Dense(units = total_words, activation = 'softmax')(x)

model1 = Model(text_in, text_out)

model1.compile(optimizer = RMSprop(lr = 0.001), 
               loss = 'categorical_crossentropy', 
               metrics = ['accuracy'])

model1.summary()

Running model...
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, None, 100)         389300    
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 3)           1248      
_________________________________________________________________
bidirectional_8 (Bidirection (None, 6)                 168       
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 3893)              27251     
Total params: 417,967
Trainable params: 417,967
Non-trainable params: 0
____________________________________

In [48]:
epochs = 100
batch_size = 32

model1.fit(X, y, epochs = epochs, batch_size = batch_size, shuffle = True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





13185/13185 [==============================] - 24s 2ms/step - loss: 7.4365 - acc: 0.2168
Epoch 2/100
13185/13185 [==============================] - 21s 2ms/step - loss: 6.2697 - acc: 0.2176
Epoch 3/100
13185/13185 [==============================] - 20s 2ms/step - loss: 6.2046 - acc: 0.2176
Epoch 4/100
13185/13185 [==============================] - 20s 2ms/step - loss: 6.1629 - acc: 0.2176
Epoch 5/100
13185/13185 [==============================] - 20s 1ms/step - loss: 6.1325 - acc: 0.2176
Epoch 6/100
13185/13185 [==============================] - 20s 2ms/step - loss: 6.0802 - acc: 0.2176
Epoch 7/100
13185/13185 [==============================] - 20s 2ms/step - loss: 6.0282 - acc: 0.2210
Epoch 8/100
13185/13185 [==============================] - 20s 2ms/step - loss: 5.9413 - acc: 0.2491
Epoch 9/100
13185/13185 [==============================] - 20s 2ms/step - loss: 5.8473 - a

In [0]:
def sample_with_temp(preds, temperature = 1.0):

    preds = np.array(preds).astype('float64')
    exp_preds = np.exp( np.log(preds) / temperature ) 
    preds = exp_preds / np.sum(exp_preds)
    probability = np.random.multinomial(1,preds,1)

    return np.argmax(probability)

def generate_text(seed_text, next_words, model, max_sequence_len=20 , temperature = 1.0):
    output_text = seed_text
    seed_text = start_story + seed_text
    #print(seed_text)

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        #print(token_list)
        #print(type(token_list))
        token_list = np.reshape(token_list, (1, max_sequence_len))

        probs = model.predict(token_list, verbose = 0)[0]
        y_index = sample_with_temp(probs, temperature = temperature)

        output_word = tokenizer.index_word[y_index] if y_index > 0 else ''

        if output_word == '|':
            #break
            continue

        seed_text += output_word + ' '
        output_text += output_word + ' '

    return output_text

In [55]:
print(generate_text("", 50, model1))

아냐 br 게 br 내가 나도 몰라 br 아 자유롭고 네 아마 이제 네게 없어 br 친구들은 당신은 br 난 난 br 난 난 없어 또 br 시간 건 몰라 br 에스키모 br 나도 내 해 br 난 없는 ' br 난 해 해 br 노래가 어디로 br 난 없는 


In [0]:
model1.save(root_dir + '2Floor_Bidirectional_Many-to-One.h5')